In [ ]:
from llama_index.core import StorageContext, load_index_from_storage
from constants import embed_model

storage_context = StorageContext.from_defaults(persist_dir='index/')
index = load_index_from_storage(storage_context, embed_model=embed_model)

In [ ]:
from llama_index.core.tools import QueryEngineTool
from constants import llm_model 

query_engine = index.as_query_engine(llm=llm_model, simularity_top_k=5)

rag_tool = QueryEngineTool.from_defaults(
  query_engine,
  name='research_paper_query_engine_tool',
  description='A RAG tool for querying research papers'
)

In [ ]:
from IPython.display import Markdown, display

def display_prompt_dict(prompts_dict):
    for key, prompt in prompts_dict.items():
        display(Markdown(f"**Prompt key**: {key}"))
        print(prompt.get_template())

In [ ]:
prompt_dict = query_engine.get_prompts()
display_prompt_dict(prompt_dict)

In [ ]:
from tools import download_pdf, fetch_arxiv_papers
from llama_index.core.tools import FunctionTool

download_pdf_tool = FunctionTool.from_defaults(download_pdf, name='download_pdf_file_tool', description='A tool to download pdfs')
fetch_arxiv_papers_tool = FunctionTool.from_defaults(fetch_arxiv_papers, name='fetch_arxiv_papers_tool', description='A tool to fetch arxiv papers')


In [ ]:
from llama_index.core.agent import ReActAgent
from constants import llm_model 

agent = ReActAgent.from_tools(
  [rag_tool, download_pdf_tool, fetch_arxiv_papers_tool],
  name='research_paper_query_tool',
  description='An agent for querying research papers',
  llm=llm_model,
  verbose=True
)

In [ ]:
query_template = """I am interested in the topic of {topic}.
Find papers in your knowledge databse to this topic.
Use the following template to query research_paper_query_tool tool:
'Provide title, summary, authers and link to download for papers related to {topic}'
If there are not, could you fetch the recent one from arxiv
"""

In [ ]:
answer = agent.chat(query_template.format(topic='Role Playing Games'))

In [ ]:
from llama_index.core.tools import QueryEngineTool
from constants import llm_model 

query_engine = index.as_query_engine(llm=llm_model, simularity_top_k=5)

result = query_engine.query('How can LLM be used in a role playing game')

print(result)